In [1]:
import ast

# Input string
input_string = "[{'HumanMessage': {'content': 'hi'}, 'AIMessage': {'content': 'Echo: ไม่เป็นไรค่ะ มีอะไรให้ช่วยไหมค่ะ?'}}]"

# Convert the string to a list
output_list = ast.literal_eval(input_string)

print(output_list)
print(type(output_list))


In [4]:
import json

# Step 1: Parse the JSON string to a Python list of dictionaries
try:
    latestMemory = json.loads(latestMemory_string)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    latestMemory = []

# Step 2: Extract and print the content as "Human: text" and "AI: text"
for message in latestMemory:
    try:
        human_content = message["HumanMessage"]["content"]
        ai_content = message["AIMessage"]["content"]
        print(f"Human: {human_content}")
        print(f"AI: {ai_content}")
    except KeyError as e:
        print(f"Missing expected key: {e}")
    except TypeError as e:
        print(f"Unexpected data format: {e}")


In [5]:
import torch

In [6]:
torch.__version__

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
x = torch.rand(5, 3).to(device)

In [9]:
x

# K-Means

In [10]:
import matplotlib.pyplot as plt

In [11]:
N = 100
x1 = torch.randn(N, 2)
x2 = torch.randn(N, 2) + torch.Tensor([3, 3])
x3 = torch.randn(N, 2) + torch.Tensor([-3, 3])

In [12]:
X = torch.cat([x1, x2, x3], dim=0)
print(X.shape)

In [13]:
for i in range(3):
    plt.scatter(X[i*N:(i+1)*N, 0], X[i*N:(i+1)*N, 1])
plt.show()

In [40]:
K = 3
centroids = X[torch.randperm(len(X))[:K]]
while True:
    D = torch.cdist(X, centroids)
    clusters = torch.argmin(D, dim=1)
    for i in range(3):
        plt.scatter(X[i*N:(i+1)*N, 0], X[i*N:(i+1)*N, 1])
    old_centroids = centroids.clone()
    for k in range(K):
        plt.scatter(centroids[k, 0], centroids[k, 1], marker='x')
        centroids[k] = X[clusters == k].mean(dim=0)
    change = torch.sum((centroids - old_centroids) ** 2)
    if change < 0.1:
        break
    plt.show()

In [15]:
import time
device = 'cuda'

N = 100000
x1 = torch.randn(N, 2)
x2 = torch.randn(N, 2) + torch.Tensor([3, 3])
x3 = torch.randn(N, 2) + torch.Tensor([-3, 3])
X = torch.cat([x1, x2, x3], dim=0).to(device)

t1 = time.time()
K = 3
centroids = X[torch.randperm(len(X))[:K]]
for _ in range(100):
    D = torch.cdist(X, centroids)
    clusters = torch.argmin(D, dim=1)
    for k in range(K):
        centroids[k] = X[clusters == k].mean(dim=0)
print(time.time() - t1)

# Auto differentiation

In [62]:
x = torch.tensor(3., requires_grad=True)
y = x**2 + 2*x + 1 
# y' = 2x + 2
y.backward()

In [65]:
x.grad # 2(3) + 2

# Regression

In [66]:
N = 100
x = torch.rand(N, 1)
y = torch.sin(3 * x + 1) + 0.5 * torch.rand(N, 1)
plt.scatter(x, y)

In [68]:
ones = torch.ones(N, 1)
X = torch.cat([x, ones], dim=1)

In [74]:
R = torch.inverse(X.T @ X) @ X.T @ y

In [76]:
x_ = torch.linspace(0, 1, 100).reshape(-1, 1)

In [82]:
y_ = x_ * R[0] + R[1]

In [83]:
plt.scatter(x, y)
plt.scatter(x_, y_, color='r')
plt.show()

nonlinear

In [84]:
ones = torch.ones(N, 1)
X = torch.cat([x**2, x, ones], dim=1)

In [86]:
R = torch.inverse(X.T @ X) @ X.T @ y

In [88]:
y_ = x_**2 * R[0] + x_ * R[1] + R[2]

In [89]:
plt.scatter(x, y)
plt.scatter(x_, y_, color='r')
plt.show()

In [134]:
device = 'cpu'

N = 100000000
x = torch.rand(N, 1).to(device)
y = torch.sin(3 * x + 1) + 0.5 * torch.rand(N, 1).to(device)

t1 = time.time()
degree = 1
ones = torch.ones(N, 1).to(device)
X = torch.cat([x, ones], dim=1)
for i in range(2, degree + 1):
    X = torch.cat([x**i, X], dim=1)
R = torch.inverse(X.T @ X) @ X.T @ y
print(time.time() - t1)

# MLP

In [135]:
from torch import nn

In [136]:
# [1] 32 16 [1]
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return self.fc3(x)

In [140]:
(1*32+32) + (32*16+16) + (16*1+1)

In [141]:
model = Regressor()

In [146]:
gen = iter(model.parameters())

In [152]:
next(gen).shape

In [155]:
N = 1000
x = torch.rand(N, 1).to(device)
y = torch.sin(3 * x + 1) + 0.5 * torch.rand(N, 1).to(device)

In [174]:
device = 'cuda' 

model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

model.train()
for epoch in range(5000):
    y_ = model(x.to(device))
    loss = criterion(y_, y.to(device))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'{epoch}: {loss.item()}')

In [177]:
x_ = torch.linspace(0, 1, 100).reshape(-1, 1)
model.eval()
with torch.no_grad():
    y_ = model(x_.to(device))
plt.scatter(x, y)
plt.scatter(x_, y_.cpu(), color='r')
plt.show()

# Classification

In [182]:
N

In [181]:
X.shape

In [183]:
Y = torch.cat([torch.zeros(N), torch.ones(N), torch.ones(N) * 2]).long()

In [184]:
Y

In [185]:
X.shape[1]

In [187]:
len(Y.unique())

In [188]:
# [2] 32 16 [3]
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 3)
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return torch.sigmoid(self.fc3(x))

In [190]:
model = Classifier()

In [192]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

model.train()
for epoch in range(5000):
    y_ = model(X.to(device))
    loss = criterion(y_, Y.to(device))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'{epoch}: {loss.item()}')

In [195]:
model.eval()
with torch.no_grad():
    y_ = model(X.to(device))
acc = torch.sum(Y == y_.argmax(dim=1).cpu()) / len(Y)
print(acc.item())

In [2]:
from sklearn.datasets import load_iris
import numpy as np
df = load_iris(as_frame=True)

In [3]:
itrain = np.r_[0:25, 50:75, 100:125]
itest = np.r_[25:50, 75:100, 125:150]
xtrain = df['data'].iloc[itrain].values
xtest = df['data'].iloc[itest].values
ytrain = df['target'].iloc[itrain].values
ytest = df['target'].iloc[itest].values

In [4]:
xtrain.shape

In [4]:
len(np.unique(ytrain))

In [1]:
from torch import nn
import torch
m = nn.LogSigmoid()
input = torch.randn(2)
input

In [9]:

output = m(input)
output

In [60]:
t = torch.rand(4, 4)
b = t.view(2, 8)
t.storage().data_ptr() == b.storage().data_ptr()  # `t` and `b` share the same underlying data.


In [64]:
b[0][0] = 3.14
t[0][0] # เปลี่ยนตาม

In [5]:
# [4] 128 [3]
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 3)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = nn.functional.leaky_relu(self.fc2(x))
        return torch.softmax(self.fc3(x), dim=1)

In [6]:
xtrain_ = torch.tensor(xtrain, dtype=torch.float32)

In [7]:
xtest_ = torch.tensor(xtest, dtype=torch.float32)

In [8]:
ytrain_ = torch.tensor(ytrain, dtype=torch.long)

In [9]:
ytest_ = torch.tensor(ytest, dtype=torch.long)

In [10]:
model = Classifier()

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(2000):
    y_ = model(xtrain_.to(device))
    loss = criterion(y_, ytrain_.to(device))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 500 == 0:
        print(f'{epoch}: {loss.item()}')

In [12]:
model.eval()
with torch.no_grad():
    y_ = model(xtest_.to(device))
acc = torch.sum(ytest_ == y_.argmax(dim=1).cpu()) / len(y_)
print(acc.item())

In [13]:
torch.save(model.state_dict(), 'model.pth') 

In [293]:
model.state_dict()

In [296]:
df['data'].columns

In [297]:
df['data'].max() # สำหรับ ทำ st.slider หรืออื่นๆ เพื่อดู scale ของค่าที่มีจาก data

## new dataset with dataset not just number

In [2]:
import seaborn as sns

In [3]:
df = sns.load_dataset('titanic')
df

In [4]:
df.nunique() # แต่ละ column มีกี่ค่า 

In [5]:
df = df.drop(columns=['class', 'adult_male', 'deck', 'embark_town', 'alive', 'alone'])
# แต่มี embarked ก็รู้ embark_town เลย drop ก่อน

In [6]:
import pandas as pd

In [7]:
df = pd.get_dummies(df, dtype=float) # มันจะเป็น one-hot ออกมา
# เราควรเก็บ feature ที่รู้อันหนึ่ง แต่เดาอีกอันไม่ได้ เช่น embarked_
# แต่พวก sex_female = 0 ก็รู้ว่า มันเป็น male ไม่ควรเอาเข้าเพื่อทำ classify

In [8]:
df  

In [9]:
df = df.drop(columns=['sex_female'])

In [10]:
df

In [11]:
df = df.dropna() # impute 

In [12]:
X = df.iloc[:, 1:] # คือเอาทุก column แต่เอาตั้งแต่ column 1 ไม่เอา target (column survived)
Y = df.iloc[:, 0]

In [13]:
X # สังเกตว่าค่ามันโดด เราควรต้อง Norm มันด้วย

In [14]:
Y

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2) # ทดสอบ performance model ก็เลยแบ่ง

In [17]:
xtrain = torch.tensor(xtrain.values, dtype=torch.float32)
xtest = torch.tensor(xtest.values, dtype=torch.float32)
ytrain = torch.tensor(ytrain.values, dtype=torch.float32)
ytest = torch.tensor(ytest.values, dtype=torch.float32)
ytrain = ytrain.reshape(-1, 1)
ytest = ytest.reshape(-1, 1)

In [18]:
xtrain.shape

In [19]:
ytrain.unique()

In [23]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(12, 128)
        # self.fc2 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(128, 1) # เราไม่นิยม ถ้า มี 2 class เราจะใช้เป็น 1 แทน (dummy variable problem) # เพราะยังไง เราก็รู้ อีกอัน และเป็นแบบนี้จะใช่ softmax ไม่ได้ เพราะมันดูหลาย class ทำให้ตัว loss ใช้ CrossEntropyloss ไม่ได้ (เพราะเหมาะกับ3 ขึ่นไป) ซึ่ง 0-1 มันเป็น float ก็ต้อง ทำตัว train, test ให้เป็น type เดียวกัน
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        # x = nn.functional.leaky_relu(self.fc2(x))
        return torch.sigmoid(self.fc2(x)) # sigmoid rank ค่า 0-1

In [24]:
model = Classifier()

In [25]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)

criterion = nn.BCELoss() # -> binary cross entropy loss => 0-1
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

model.train()
for epoch in range(2000):
    y_ = model(xtrain.to(device))
    loss = criterion(y_, ytrain.to(device))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 500 == 0:
        print(f'{epoch}: {loss.item()}')

In [26]:
model.eval()
with torch.no_grad():
    y_ = model(xtest.to(device))
acc = torch.sum(ytest == y_.round().cpu()) / len(y_)
print(acc.item())

In [27]:
torch.save(model.state_dict(), 'model_titanic_cls.pth') # ทำ widget ให้เหมาะ บน streamlit แล้ว